In [9]:
from flask import request, jsonify
from flask_cors import cross_origin
from pymongo import MongoClient
from bson import ObjectId
from flask import Blueprint

import os
import pickle
import random
import pandas as pd
import numpy as np
from pymongo import MongoClient, GEO2D
from bson import ObjectId
import os
import dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from utils.models import unique
import pandas as pd
import pyodbc
import psycopg2
import sys
import os
from dotenv import load_dotenv
load_dotenv()
import json
import pickle
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def recsys():
# user info

    user  =  '5fc76f2f4b1cab6b736b1019'
    lat = 6.524379
    long = 3.379206
    
    # impute user id and location
    # userid =  '5fc76f2f4b1cab6b736b1019'
    # lat = 6.524379
    # long = 3.379206

    # get the permission link
    STAGING_URI = os.getenv("STAGING_URI")
    PROD_DB = os.getenv("PROD_DB")
    DEV_URI = os.getenv("MONGO_LINK")


    # initialize connection to MongoDB
    try:
        conn = MongoClient(PROD_DB)
        db = conn['odadb']
        print(db)
    except Exception as p:

        print('''jsonify({"msgcode": 4, "status": "failed", "message": "Failed to Connect To Mongo Schema {}".format(p)}), 401''')




    # Get orders
    orders = pd.DataFrame((list(db.orders.aggregate([{"$unwind": "$products"}]))));
    orders['products_id'] = orders['products'].apply(lambda x: x.get("productId"))
    orders['productName'] = orders['products'].apply(lambda x: x.get("productName"))
    orders['genericProductId'] = orders['products'].apply(lambda x: x.get("genericProductId"))
    orders['quantity'] = orders['products'].apply(lambda x: x.get("quantity"))
    orders['price'] = orders['products'].apply(lambda x: x.get("price"))
    orders['buyerId'] = orders['buyerId'].astype(str)
    orders = orders[
        ['buyerDropOffHandShake', 'buyerId', 'orderId', 'products_id', 'productName', 'genericProductId',
         'quantity', 'price']]
    orders['productName'] = orders['productName'].astype(str)
    orders['buyerId'] = orders['buyerId'].astype(str)
    # orders.dropna(subset=['productName'], how='all', inplace=True)
    orders = orders.dropna(subset=['productName'])
    orderss = orders.drop(['buyerDropOffHandShake'], axis=1)

    userid = str(user)
    useritems = orderss[orderss['buyerId'] == userid]
    # Get Storeproducts
    # storeproducts = list(db.storeproducts.find())
    # storeproducts= pd.DataFrame(list(storeproducts))

    storeproducts = db.storeproducts.find({"status": "ACTIVE", "storeCoord": {
        "$geoWithin": {"$centerSphere": [[float(long), float(lat)], 4 / 6378.1]}}},
                                          {'image': 1, 'genericProductId': 1, 'productName': 1,
                                           'description': 1, 'storeCoord': 1})
    storeproducts = pd.DataFrame(list(storeproducts))

    # Get  genericproducts
    genericproducts = list(db.genericproducts.find())
    genericproducts = pd.DataFrame(list(genericproducts))

    if len(storeproducts) < 1:

        print('''return jsonify(
            {"msgcode": 6, "status": "success", "message": " No related product found nearby", "data": []}), 200''')
    elif len(useritems) < 1:

        print('''return jsonify({"msgcode": 6, "status": "success", "message": " No order history of customer found",
                        "data": []}), 200''')

    # selected generic product columns is named as data for training
    data = genericproducts[
        ['subcategoryName', 'productName', 'description', 'categoryName', 'comment', 'image']]

    # preprocess data(genericproducts)
    data['subcategoryName'] = data['subcategoryName'].astype(str)
    data['productName'] = data['productName'].astype(str)
    data['description'] = data['description'].astype(str)
    data['subcategoryName'] = data['subcategoryName'].str.strip('[')
    data['subcategoryName'] = data['subcategoryName'].str.strip(']')

    # data = data.drop_duplicates()
    data.dropna(subset=['productName', 'description'], how='all', inplace=True)
    data = data.reset_index()

    # preprocess storeproducts and get needed columns
    storeproducts.columns
    storeproducts = storeproducts[['storeCoord', 'image', 'productName', 'description', 'genericProductId']]

    print("orders.shape :", orders.shape)
    print("storeproducts :", storeproducts.shape)
    print("genericproducts :", data.shape)

    # vectorize data description columns

    tfv = TfidfVectorizer(min_df=3, max_features=None,
                          strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                          ngram_range=(1, 3),
                          stop_words='english')

    tfv_matrix = tfv.fit_transform(data['description'])

    # tfv_matrix.shape

    # Compute the sigmoid kernel to get correlations amongs items
    sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

    # Reverse mapping of indices and productName
    indices = pd.Series(data.index, index=data['productName'])

    def give_rec(items, sig=sig):
        # Get the index corresponding to productName
        idx = indices[items]

        # Get the pairwsie similarity scores
        sig_scores = list(enumerate(sig[idx]))

        # Sort the items
        sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

        # Scores of the 500 most similar items
        sig_scores = sig_scores[1:500]

        # item indices
        m_indices = [i[0] for i in sig_scores]

        # Top most similar items
        return data['productName'].iloc[m_indices]

    storeproducts.columns

    # impute user id and location
    # userid =  '5fc76f2f4b1cab6b736b1019'
    # lat = 6.524379
    # long = 3.379206

    # remove duplicates
    orderss.drop_duplicates(keep=False, inplace=True)

    # get the a random product name of one of identified customer's orders

    # give recommendation based on past buys

    one_product = orderss[orderss['buyerId'] == user].productName.sample().values[0]
    print(one_product)
    
    if one_product is None  or  one_product == None :
        print('''return  jsonify({"msgcode": 5, "status": "failed", "message": "No recommendation"}), 400''')
    
    else :

        recomm = give_rec(one_product)

        # merging recommended items with storeproducts filtered by nearest locations
        recomm = pd.DataFrame(recomm, columns=['productName'])
        storeproducts1 = storeproducts[['image', 'productName', 'description', 'genericProductId']]
        storeproducts1['productName'] = storeproducts1['productName'].astype(str)
        recomm_final = recomm.merge(storeproducts1, how='inner', on='productName')

        final = pd.DataFrame()
        final[['image', 'productName', 'uid']] = recomm_final[['image', 'productName', 'genericProductId']]
        final = final.to_dict(orient='records')
        final = unique(final)


        print(final)

        print('''return jsonify({"msgcode": 8, "status": "success", "data": final}), 200''')



        print('''return jsonify({"msgcode": 10, "status": "success", "message": 'Something went Wrong'}), 500''')

    print(final)